<a href="https://colab.research.google.com/github/jadenfang/Elden-Ring-Sentiment-Analysis/blob/main/Sentiment_Analysis_of_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from gensim.models import Word2Vec
import numpy as np
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [60]:
df = pd.read_csv("elden_ring_steam_reviews.csv")

df = df[df["language"] == "english"]

df["label"] = df["voted_up"].astype(int)

df = df.dropna(subset=["review", "label"])

texts = df["review"].astype(str).tolist()
labels = df["label"].tolist()

sentences = list(zip(texts, labels))

print("Total samples:", len(sentences))
print("Label distribution:")
print(df["label"].value_counts())
print("Example review:", sentences[0])

X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    texts, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)


Total samples: 9765
Label distribution:
label
1    9174
0     591
Name: count, dtype: int64
Example review: ('being killed over and over again is fun', 1)


In [61]:
# ----------------------
# Preprocessing
# ----------------------
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    return text.split()

In [62]:
corpus = [tokenize(text) for text in X_train_texts]
print(corpus)

[['goood', 'game'], ['amazing', 'game', 'surprises', 'you', 'constantly', 'there', 'is', 'a', 'ton', 'of', 'content', 'that', 'is', 'easily', 'missed', 'so', 'there', 'is', 'high', 'replay', 'value'], ['historically', 'accurate', 'to', 'how', 'my', 'dad', 'walked', 'to', 'school', 'good', 'job'], ['from', 'the', 'developers', 'of', 'dark', 'souls', 'more', 'dark', 'souls', 'no', 'disappointment', 'anyway', 'head', 'into', 'dark', 'souls', 'before', 'elden', 'ring', 'make', 'your', 'own', 'opinion', 'on', 'this', 'game', 'dont', 'read', 'reviews', 'that', 'nitpick', 'stuff', 'and', 'turns', 'you', 'sad', 'off', 'the', 'game', 'you', 'wanna', 'try', 'out', 'udont', 'spoil', 'it', 'for', 'yourselfu', 'all', 'dark', 'souls', 'have', 'their', 'own', 'strengths', 'as', 'devs', 'try', 'out', 'different', 'approaches'], ['a', 'true', 'work', 'of', 'art'], ['the', 'gus', 'are', 'worth', 'the', 'pain'], ['mhmm', 'yumyum'], ['hit', 'and', 'die'], ['when', 'my', 'computer', 'is', 'not', 'exploding

In [63]:
# ----------------------
# Train Word2Vec
# ----------------------
model = Word2Vec(corpus, vector_size=50, window=4, min_count=1, sg=1)

In [64]:
# ----------------------
# Sentence vector
# ----------------------
def sentence_vector(tokens):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    if not vecs:
        return np.zeros(model.vector_size)
    return np.mean(vecs, axis=0)

In [65]:
X_train = np.array([sentence_vector(tokenize(t)) for t in X_train_texts])
X_test  = np.array([sentence_vector(tokenize(t)) for t in X_test_texts])

In [66]:
# ----------------------
# Simple sentiment prototypes
# ----------------------
y_train = np.array(y_train)

pos_vec = np.mean(X_train[y_train == 1], axis=0)
neg_vec = np.mean(X_train[y_train == 0], axis=0)

In [67]:
def cosine(a, b):
    na = np.linalg.norm(a)
    nb = np.linalg.norm(b)
    if na == 0 or nb == 0:
        return -1  # force "far away"
    return np.dot(a, b) / (na * nb)

def predict(sentence):
    v = sentence_vector(tokenize(sentence))
    return "positive" if cosine(v, pos_vec) > cosine(v, neg_vec) else "negative"

def predict_vector(v):
    return 1 if cosine(v, pos_vec) > cosine(v, neg_vec) else 0

In [68]:
# ----------------------
# Try it
# ----------------------
tests = [
    "I love dying over and over",
    "This game is too hard",
    "I have 1000 hours and still cant beat this boss",
    "I hate this game"
]

for t in tests:
    print(t, "→", predict(t))

valid_idx = [i for i, v in enumerate(X_test) if np.linalg.norm(v) != 0]

X_test_valid = X_test[valid_idx]
y_test_valid = [y_test[i] for i in valid_idx]

y_pred = [predict_vector(v) for v in X_test_valid]

print("Accuracy:", accuracy_score(y_test_valid, y_pred))


I love dying over and over → negative
This game is too hard → positive
I have 1000 hours and still cant beat this boss → negative
I hate this game → positive
Accuracy: 0.5402792696025779
